In [1]:
import calendar
import pandas as pd
import numpy as np
import panel as pn
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import show
pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
today

datetime.date(2022, 5, 29)

### Tables in the process

In [2]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trans name spd reason qty target current percent active xdate'.split()

format_dict = {
    'qty':'{:,}','available_qty':'{:,}',
    'price':'{:.2f}','cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}',
    'buy_target':'{:.2f}','sell_target':'{:.2f}','target':'{:.2f}','current':'{:.2f}',
    'beta':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}','volume':'{:,.2f}',
    'percent':'{:,.2f}',
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
}

In [3]:
sql = '''
SELECT *
FROM orders
ORDER BY id DESC
LIMIT 1'''
orders = pd.read_sql(sql, conlite)
orders

,id,trade,name,qty,price,active,reason,market,xdate
0,19,S,WHART,10000,11.1,0,P50%,SET,2022-05-30


In [4]:
names = orders['name']
name = names.to_string(index=False)

In [5]:
sql = '''
SELECT *
FROM dividend
WHERE name = '%s'
'''
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,WHART,0.1195,0.2553,0.1915,0.1915,0.7578,"30,000",2022-05-30,2022-06-15,,0


In [6]:
sql = """
SELECT * FROM stocks WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * FROM stocks WHERE name = 'WHART'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,687,WHART,14.08,10.80,I,0.00,11.10,24.74,0.21,11.70,"10,000",-5,4,"30,000",0,0,P50%,SET


### Create orders from stocks after adjust stocks in port_lite from consensus

In [7]:
sql = '''
DELETE FROM orders'''
rp = conlite.execute(sql)
rp.rowcount

19

In [8]:
sql = '''
SELECT status AS trade, name, qty, buy_target AS price, 0 AS active, reason, market
FROM stocks
WHERE status IN ('B','O')
ORDER BY name'''
df_buy = pd.read_sql(sql, conlite)
df_buy

,trade,name,qty,price,active,reason,market
0,O,AIT,18000,6.75,0,P25%,SET
1,O,AP,10000,10.50,0,XXX,SET100
2,O,ASK,3000,40.00,0,XXX,SET
3,O,BANPU,12000,12.00,0,XXX,SET50
4,O,BCPG,12000,11.50,0,3L,SET100
5,B,BDMS,1500,24.40,0,RD05pct,SET50
6,O,BPP,9000,15.60,0,XXX,SET100
7,O,GLOBAL,9000,20.00,0,,SET50
8,O,GULF,6000,46.00,0,1L,SET50
9,O,KKP,3000,69.00,0,3L,SET100


In [9]:
sql = '''
SELECT status AS trade, name, qty, sell_target AS price, 0 AS active, reason, market
FROM stocks
WHERE status IN ('I','S')'''
df_sell = pd.read_sql(sql, conlite)
df_sell

,trade,name,qty,price,active,reason,market
0,I,NER,9000,7.0,0,P50%,SET
1,I,IP,2500,20.3,0,CP1S,mai
2,I,WHART,10000,11.1,0,P50%,SET


In [10]:
df_order = df_buy.append(df_sell, ignore_index=True)
df_order.shape

(19, 7)

In [11]:
df_order.groupby(['trade']).count()

,name,qty,price,active,reason,market
trade,,,,,,
B,2,2,2,2,2,2
I,3,3,3,3,3,3
O,14,14,14,14,14,14


In [12]:
df_order.loc[df_order.trade == 'I','trade'] = 'S'
df_order.groupby(['trade']).count()

,name,qty,price,active,reason,market
trade,,,,,,
B,2,2,2,2,2,2
O,14,14,14,14,14,14
S,3,3,3,3,3,3


In [13]:
sql = '''
SELECT name, xdate
FROM dividend
'''
dividend = pd.read_sql(sql, const)
dividend.columns = dividend.columns.str.lower()
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend.shape

(83, 2)

In [14]:
df_merge = pd.merge(df_order, dividend, on='name', how='outer', indicator=True)
df_merge.shape

(91, 9)

### If there is no dividend record, must create dividend record before proceed

In [15]:
no_div = df_merge[df_merge['_merge'] == 'left_only']
no_div

,trade,name,qty,price,active,reason,market,xdate,_merge
0,O,AIT,18000.0,6.75,0.0,P25%,SET,NaT,left_only
4,O,BCPG,12000.0,11.50,0.0,3L,SET100,NaT,left_only
6,O,BPP,9000.0,15.60,0.0,XXX,SET100,NaT,left_only
11,O,SIS,3600.0,28.75,0.0,6L,SET,NaT,left_only
12,O,SPRC,15000.0,11.50,0.0,XXX,SET100,NaT,left_only
13,O,SSP,10000.0,9.50,0.0,6L,SET,NaT,left_only
14,O,TOP,3600.0,56.00,0.0,1L,SET50,NaT,left_only
15,O,TTB,150000.0,1.20,0.0,XXX,SET50,NaT,left_only


### Rerun from here

In [16]:
# Include both having diviedend and having no dividend records
mask = ((df_merge['_merge'] == 'both') | (df_merge['_merge'] == 'left_only'))
df_orders = df_merge[mask].copy()
df_orders.drop(['_merge'], axis=1, inplace=True)
df_orders['qty'] = df_orders['qty'].astype('int64')
df_orders['active'] = df_orders['active'].astype('int64')
df_orders['xdate'] = df_orders['xdate'].astype('str')
df_orders.groupby('trade').count()

,name,qty,price,active,reason,market,xdate
trade,,,,,,,
B,2,2,2,2,2,2,2
O,14,14,14,14,14,14,14
S,3,3,3,3,3,3,3


### Set xdate 2 weeks window of first Tuesday and second Friday

In [17]:
fm_date = '2022-05-03'

### Restart and Run All Cells

In [18]:
fltr_sts_o = (df_orders.trade == 'O') 
df_sts_o = df_orders[fltr_sts_o]
df_sts_o

,trade,name,qty,price,active,reason,market,xdate
0,O,AIT,18000,6.75,0,P25%,SET,NaT
1,O,AP,10000,10.50,0,XXX,SET100,2022-05-10
2,O,ASK,3000,40.00,0,XXX,SET,2022-03-03
3,O,BANPU,12000,12.00,0,XXX,SET50,2022-04-11
4,O,BCPG,12000,11.50,0,3L,SET100,NaT
6,O,BPP,9000,15.60,0,XXX,SET100,NaT
7,O,GLOBAL,9000,20.00,0,,SET50,2022-02-02
8,O,GULF,6000,46.00,0,1L,SET50,2022-03-04
9,O,KKP,3000,69.00,0,3L,SET100,2022-04-28
11,O,SIS,3600,28.75,0,6L,SET,NaT


In [19]:
fltr_sts_b = (df_orders.trade == 'B') 
df_sts_b = df_orders[fltr_sts_b]
df_sts_b

,trade,name,qty,price,active,reason,market,xdate
5,B,BDMS,1500,24.4,0,RD05pct,SET50,2022-03-09
10,B,ORI,15000,10.2,0,RD05pct,SET100,2022-05-09


In [20]:
fltr_sts_s = (df_orders.trade == 'S')
df_sts_s = df_orders[fltr_sts_s]
df_sts_s.sort_values('name',ascending=True)

,trade,name,qty,price,active,reason,market,xdate
17,S,IP,2500,20.3,0,CP1S,mai,2022-02-02
16,S,NER,9000,7.0,0,P50%,SET,2022-04-19
18,S,WHART,10000,11.1,0,P50%,SET,2022-05-30


In [21]:
frames = [df_sts_b, df_sts_o, df_sts_s]
df_orders = pd.concat(frames, sort=False)
df_orders.sort_values(['trade','name'],ascending=True)

,trade,name,qty,price,active,reason,market,xdate
5,B,BDMS,1500,24.40,0,RD05pct,SET50,2022-03-09
10,B,ORI,15000,10.20,0,RD05pct,SET100,2022-05-09
0,O,AIT,18000,6.75,0,P25%,SET,NaT
1,O,AP,10000,10.50,0,XXX,SET100,2022-05-10
2,O,ASK,3000,40.00,0,XXX,SET,2022-03-03
3,O,BANPU,12000,12.00,0,XXX,SET50,2022-04-11
4,O,BCPG,12000,11.50,0,3L,SET100,NaT
6,O,BPP,9000,15.60,0,XXX,SET100,NaT
7,O,GLOBAL,9000,20.00,0,,SET50,2022-02-02
8,O,GULF,6000,46.00,0,1L,SET50,2022-03-04


In [22]:
df_orders.loc[df_orders.trade == 'O','trade'] = 'B'
df_orders.groupby(['trade']).count()

,name,qty,price,active,reason,market,xdate
trade,,,,,,,
B,16,16,16,16,16,16,16
S,3,3,3,3,3,3,3


In [23]:
rcds = df_orders.values.tolist()
len(rcds)

19

In [24]:
for rcd in rcds:
    print(rcd)

['B', 'BDMS', 1500, 24.4, 0, 'RD05pct', 'SET50', '2022-03-09']
['B', 'ORI', 15000, 10.2, 0, 'RD05pct', 'SET100', '2022-05-09']
['B', 'AIT', 18000, 6.75, 0, 'P25%', 'SET', 'NaT']
['B', 'AP', 10000, 10.5, 0, 'XXX', 'SET100', '2022-05-10']
['B', 'ASK', 3000, 40.0, 0, 'XXX', 'SET', '2022-03-03']
['B', 'BANPU', 12000, 12.0, 0, 'XXX', 'SET50', '2022-04-11']
['B', 'BCPG', 12000, 11.5, 0, '3L', 'SET100', 'NaT']
['B', 'BPP', 9000, 15.6, 0, 'XXX', 'SET100', 'NaT']
['B', 'GLOBAL', 9000, 20.0, 0, '', 'SET50', '2022-02-02']
['B', 'GULF', 6000, 46.0, 0, '1L', 'SET50', '2022-03-04']
['B', 'KKP', 3000, 69.0, 0, '3L', 'SET100', '2022-04-28']
['B', 'SIS', 3600, 28.75, 0, '6L', 'SET', 'NaT']
['B', 'SPRC', 15000, 11.5, 0, 'XXX', 'SET100', 'NaT']
['B', 'SSP', 10000, 9.5, 0, '6L', 'SET', 'NaT']
['B', 'TOP', 3600, 56.0, 0, '1L', 'SET50', 'NaT']
['B', 'TTB', 150000, 1.2, 0, 'XXX', 'SET50', 'NaT']
['S', 'NER', 9000, 7.0, 0, 'P50%', 'SET', '2022-04-19']
['S', 'IP', 2500, 20.3, 0, 'CP1S', 'mai', '2022-02-02']
['

In [25]:
sql = """
INSERT INTO orders (trade, name, qty, price, active,
reason, market, xdate)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)"""
print(sql)


INSERT INTO orders (trade, name, qty, price, active,
reason, market, xdate)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)


In [26]:
for rcd in rcds:
    conlite.execute(sql, rcd)

In [27]:
sql = """
SELECT * FROM orders"""
df_orders = pd.read_sql(sql, conlite)
df_orders

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,BDMS,1500,24.40,0,RD05pct,SET50,2022-03-09
1,2,B,ORI,15000,10.20,0,RD05pct,SET100,2022-05-09
2,3,B,AIT,18000,6.75,0,P25%,SET,NaT
3,4,B,AP,10000,10.50,0,XXX,SET100,2022-05-10
4,5,B,ASK,3000,40.00,0,XXX,SET,2022-03-03
5,6,B,BANPU,12000,12.00,0,XXX,SET50,2022-04-11
6,7,B,BCPG,12000,11.50,0,3L,SET100,NaT
7,8,B,BPP,9000,15.60,0,XXX,SET100,NaT
8,9,B,GLOBAL,9000,20.00,0,,SET50,2022-02-02
9,10,B,GULF,6000,46.00,0,1L,SET50,2022-03-04


In [28]:
df_orders.loc[(df_orders.xdate == 'NaT'),['xdate']] = ['2022-02-02']
df_orders

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,BDMS,1500,24.40,0,RD05pct,SET50,2022-03-09
1,2,B,ORI,15000,10.20,0,RD05pct,SET100,2022-05-09
2,3,B,AIT,18000,6.75,0,P25%,SET,2022-02-02
3,4,B,AP,10000,10.50,0,XXX,SET100,2022-05-10
4,5,B,ASK,3000,40.00,0,XXX,SET,2022-03-03
5,6,B,BANPU,12000,12.00,0,XXX,SET50,2022-04-11
6,7,B,BCPG,12000,11.50,0,3L,SET100,2022-02-02
7,8,B,BPP,9000,15.60,0,XXX,SET100,2022-02-02
8,9,B,GLOBAL,9000,20.00,0,,SET50,2022-02-02
9,10,B,GULF,6000,46.00,0,1L,SET50,2022-03-04


In [42]:
pd.read_sql_query('SELECT * FROM orders', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,1,B,BDMS,1500,24.40,0,RD05pct,SET50,2022-03-09
1,2,B,ORI,15000,10.20,0,RD05pct,SET100,2022-05-09
2,3,B,AIT,18000,6.75,0,P25%,SET,2022-02-02
3,4,B,AP,10000,10.50,0,XXX,SET100,2022-05-10
4,5,B,ASK,3000,40.00,0,XXX,SET,2022-03-03
5,6,B,BANPU,12000,12.00,0,XXX,SET50,2022-04-11
6,7,B,BCPG,12000,11.50,0,3L,SET100,2022-02-02
7,8,B,BPP,9000,15.60,0,XXX,SET100,2022-02-02
8,9,B,GLOBAL,9000,20.00,0,,SET50,2022-02-02
9,10,B,GULF,6000,46.00,0,1L,SET50,2022-03-04


In [41]:
sqlUpd = """
UPDATE orders
SET xdate = '2022-02-02'
WHERE xdate = 'NaT'"""
rp = conlite.execute(sqlUpd)
rp.rowcount

8

In [30]:
pd.read_sql_query('SELECT * FROM orders WHERE qty = 0', conlite)

,id,trade,name,qty,price,active,reason,market,xdate


In [31]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)

### Process to set target price

### 1) Set50 records

In [32]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET50" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,6,B,BANPU,12000,12.0,0,XXX,SET50,2022-04-11
1,1,B,BDMS,1500,24.4,0,RD05pct,SET50,2022-03-09
2,9,B,GLOBAL,9000,20.0,0,,SET50,2022-02-02
3,10,B,GULF,6000,46.0,0,1L,SET50,2022-03-04
4,15,B,TOP,3600,56.0,0,1L,SET50,NaT
5,16,B,TTB,150000,1.2,0,XXX,SET50,NaT


In [33]:
name = 'PTTGC'
limit = 20 # 1 month of data
sql = """
SELECT *
FROM price
WHERE name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe()

,price,maxp,minp,qty,opnp
count,20.000000,20.000000,20.000000,2.000000e+01,20.000000
mean,47.637500,48.337500,47.087500,1.974348e+07,47.700000
std,1.692621,1.620743,1.686391,1.745643e+07,1.596872
min,44.250000,45.500000,44.250000,6.865532e+06,44.750000
25%,46.500000,47.187500,46.187500,9.569791e+06,46.875000
50%,47.500000,47.875000,47.125000,1.515338e+07,47.500000
75%,48.812500,49.375000,48.187500,2.228711e+07,48.812500
max,50.250000,51.500000,50.000000,8.572728e+07,51.000000


### 2) Set100 records

In [34]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET100" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,4,B,AP,10000,10.5,0,XXX,SET100,2022-05-10
1,7,B,BCPG,12000,11.5,0,3L,SET100,NaT
2,8,B,BPP,9000,15.6,0,XXX,SET100,NaT
3,11,B,KKP,3000,69.0,0,3L,SET100,2022-04-28
4,2,B,ORI,15000,10.2,0,RD05pct,SET100,2022-05-09
5,13,B,SPRC,15000,11.5,0,XXX,SET100,NaT


In [35]:
name = 'SINGER'
limit = 60 # 1 month of data
sql = """
SELECT *
FROM price
WHERE name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe()

,price,maxp,minp,qty,opnp
count,60.000000,60.000000,60.000000,6.000000e+01,60.000000
mean,51.766667,52.683333,50.783333,5.094307e+06,51.683333
std,3.543575,3.436120,3.529977,3.562296e+06,3.536992
min,45.750000,46.500000,44.000000,1.337690e+06,45.750000
25%,49.187500,49.937500,47.937500,3.179506e+06,49.000000
50%,51.625000,52.250000,50.375000,4.541004e+06,50.875000
75%,54.750000,55.750000,53.812500,5.730057e+06,54.750000
max,58.750000,59.250000,57.500000,2.367081e+07,59.000000


### 3) Set records

In [36]:
pd.read_sql_query('SELECT * FROM orders WHERE market = "SET" ORDER BY trade, name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,3,B,AIT,18000,6.75,0,P25%,SET,NaT
1,5,B,ASK,3000,40.00,0,XXX,SET,2022-03-03
2,12,B,SIS,3600,28.75,0,6L,SET,NaT
3,14,B,SSP,10000,9.50,0,6L,SET,NaT
4,17,S,NER,9000,7.00,0,P50%,SET,2022-04-19
5,19,S,WHART,10000,11.10,0,P50%,SET,2022-05-30


In [37]:
name = 'WHART'
limit = 60 # 1 month of data
sql = """
SELECT *
FROM price
WHERE name = '%s'
ORDER BY date DESC
LIMIT %s"""
sql = sql % (name, limit)
df = pd.read_sql(sql, const)
df.describe()

,price,maxp,minp,qty,opnp
count,60.000000,60.000000,60.000000,6.000000e+01,60.000000
mean,11.248333,11.375000,11.143333,2.087892e+06,11.286667
std,0.590459,0.591644,0.582620,4.080674e+06,0.581485
min,10.400000,10.500000,10.300000,1.533010e+05,10.400000
25%,10.775000,10.875000,10.675000,5.312840e+05,10.800000
50%,11.150000,11.350000,11.100000,1.125615e+06,11.200000
75%,11.625000,11.800000,11.600000,1.778629e+06,11.700000
max,12.400000,12.500000,12.300000,2.899951e+07,12.400000


### After call Ord-log.bat

In [38]:
def categorise(row):  
    if row['trade'] == 'B':
        return 'Buy'
    else:
        return 'Sell'

In [39]:
df = pd.read_csv('../data/orders-log.csv')
df['trans'] = df.apply(lambda row: categorise(row), axis=1)

df_tab = pn.widgets.Tabulator(df[colt], layout='fit_data', width=800)
df_tab

Tabulator(layout='fit_data', value=   trans    name  ..., width=800)

In [40]:
df[colt]

,trans,name,spd,reason,qty,target,current,percent,active,xdate
0,Buy,AIT,-3,P25%,18000,6.75,6.90,+2.22%,0,NaT
1,Buy,AP,-6,XXX,10000,10.50,11.10,+0.91%,0,2022-05-10
2,Buy,ASK,-6,XXX,3000,40.00,41.50,0.00%,0,2022-03-03
3,Buy,BANPU,-1,XXX,12000,12.00,12.10,-0.82%,0,2022-04-11
4,Buy,BBL,-9,1L,2400,125.00,129.50,+1.17%,0,2022-04-22
5,Buy,BCPG,-3,3L,12000,11.50,11.80,-0.84%,0,NaT
6,Buy,BDMS,-8,RD05pct,1500,24.40,25.50,-1.92%,0,2022-03-09
7,Buy,BPP,-3,XXX,9000,15.60,15.90,-0.62%,0,NaT
8,Buy,GLOBAL,-7,NaN,9000,20.00,20.70,+0.49%,0,2022-02-02
9,Buy,GULF,-7,1L,6000,46.00,47.75,+0.53%,0,2022-03-04
